# The basics of CityJSON in Python

In this tutorial we explore what is possible with `cjio`'s Command Line Interface (CLI)
and intorduce `cjio`'s Application Programming Interface (API).

The CLI is what you use when you invoke `cjio` from the command line, such as:
```
$ cjio some_city_model.json info
```

The API is what you use from `cjio` when working with a city model in a Python script,
and load a cityjson file with the `cityjson.load()` method.

## CLI

The primary source of information on using the CLI is its help menu. It will list all
of the available *commands* and describes how to use them. For instance the
`subset` command.

In [1]:
! cjio --help

Usage: cjio [OPTIONS] INPUT COMMAND1 [ARGS]... [COMMAND2 [ARGS]...]...

  Process and manipulate a CityJSON file, and allow different outputs. The
  different operators can be chained to perform several processing in one
  step, the CityJSON model goes through the different operators.

  To get help on specific command, eg for 'validate':

      cjio validate --help

  Usage examples:

      cjio example.json info validate
      cjio example.json assign_epsg 7145 remove_textures export output.obj
      cjio example.json subset --id house12 save out.json

Options:
  --version                Show the version and exit.
  --ignore_duplicate_keys  Load a CityJSON file even if some City Objects have
                           the same IDs (technically invalid file)

  --help                   Show this message and exit.

Commands:
  assign_epsg                Assign a (new) EPSG.
  clean                      Clean = remove_duplicate_vertices +...
  compress                   Compress a CityJ

Most of the commands have detailed descriptions and *options*, which you can view with the help
menu of the particular command.

In [2]:
! cjio subset --help

Usage: cjio subset [OPTIONS]

  Create a subset of a CityJSON file. One can select City Objects by (1) IDs
  of City Objects; (2) bbox; (3) City Object type; (4) randomly.

  These can be combined, except random which overwrites others.

  Option '--exclude' excludes the selected objects, or "reverse" the
  selection.

Options:
  --id TEXT                       The ID of the City Objects; can be used
                                  multiple times.

  --bbox FLOAT...                 2D bbox: (minx miny maxx maxy).
  --random INTEGER                Number of random City Objects to select.
  --cotype [Building|Bridge|Road|TransportSquare|LandUse|Railway|TINRelief|WaterBody|PlantCover|SolitaryVegetationObject|CityFurniture|GenericCityObject|Tunnel]
                                  The City Object type
  --exclude                       Excludes the selection, thus delete the
                                  selected object(s).

  --help                          Show this message and exi

### Pipelines of commands

The cjio commands can be chained, creating a pipeline. The input citymodel is passed
from left to right, each command reading the output of the previous command.

Commands like `info` and `validate` output information in the console and just
pass the 3D city model to the next command. Other commands like `assign_epsg` and
`subset` modify the city model.

In [3]:
! cjio data/sample.json extract_lod 1.3 subset --id "NL.IMBAG.Pand.0518100000285075" info --long

Parsing data/sample.json
Extract LoD:1.3
Subset of CityJSON
{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "bbox": [
    76538.049,
    451859.682,
    0.266,
    76555.762,
    451874.674,
    11.216000000000001
  ],
  "transform/compressed": true,
  "cityobjects_total": 1,
  "cityobjects_present": [
    "Building",
    "BuildingPart"
  ],
  "materials": false,
  "textures": false,
  "vertices_total": 108,
  "geom_primitives_present": [
    "Solid"
  ],
  "level_of_detail": [
    1.3
  ],
  "semantics_surfaces_present": [
    "WallSurface",
    "RoofSurface",
    "GroundSurface"
  ],
  "cityobject_attributes": [
    "h_dak_50p",
    "status",
    "h_dak_min",
    "kas_warenhuis",
    "h_maaiveld",
    "ondergronds_type",
    "oorspronkelijkbouwjaar",
    "pw_bron",
    "val3dity_codes_lod12",
    "reconstruction_skipped",
    "reconstructie_methode",
    "rmse_lod13",
    "data_coverage",
    "lod11_replace",
    "geconstateerd",
    "h_dak_max",
    "val3dity_codes_lod13",
    "rms

### Don't forget to save

No matter what command or pipeline you use, `cjio` doesn't save the result, unless
you add the `save` command at the end. The `save` command is a special one, because
it's a terminal operator, which means that it needs to stand at the end of the pipeline.
At the moment (v0.6.10) it is not possible to save intermediate states from the
pipeline.

## CityJSON in Python without cjio

It is fully possible to work with 3D city models from CityJSON files in Python without
`cjio`'s API. In fact, this is how the CLI commands are implemented. To see it practice,
 browse through `cjio`'s source code, particularly the `cityjson` and `cjio` modules in
 https://github.com/cityjson/cjio .

As CityJSON files are just regular json files, you only need the python standard
libraries and some knowledge of the [CityJSON schema](https://www.cityjson.org/specs/latest) to get going.

In [4]:
import json
from pathlib import Path

with (Path("data") / "sample.json").resolve().open("r") as fo:
    cm = json.load(fo)

for co_id, co in cm["CityObjects"].items():
    print(f"Found CityObject {co_id} of type {co['type']}")

Found CityObject NL.IMBAG.Pand.0518100000285075 of type Building
Found CityObject NL.IMBAG.Pand.0518100000285075-0 of type BuildingPart
Found CityObject NL.IMBAG.Pand.0518100000346770 of type Building
Found CityObject NL.IMBAG.Pand.0518100000269458 of type Building
Found CityObject NL.IMBAG.Pand.0518100000346770-0 of type BuildingPart
Found CityObject NL.IMBAG.Pand.0518100000269458-0 of type BuildingPart


## API intro

cjio was developed as a CLI tool for manipulating complete citymodels from CityJSON
files. The initial goal was to test how simple is it to work with CityJSON in python,
using only the standard libraries.

The API was developed after the CLI was written, and it was only an afterthought.
The API is meant to provide functions that make it even easier to work on citymodels,
individual CityObjects in particular, from within python applications.

Therefore, there the integration between the API and CLI is not very tight at the
moment (v0.6.10).

Both CLI and API uses the `cjio.cityjson.CityJSON` class to store a city model. However, the
CLI loads the data from the cityjson file into the `CityJSON.j` class member, following
the cityjson schema exactly.
On the other hand, the API reads the contents of `CityJSON.j` and populates the
`CityJSON.cityobjects` class member with the CityObject data from the citymodel.
In this process, instances of `cjio.models.CityObject` and `cjio.models.Geometry` are
created to store the cityobject's data, including their geometry. In other words,
each `CityObject` in `CityJSON.cityobjects` stores its complete geometry with their
coordinates.

It is possible to use the CLI methods "from the API", since both of them utilizes the
common `CityJSON` class. However, if the citymodel was modified via the API methods,
then the data in `CityJSON.j` needs to be updated before calling any CLI method.
To do this, call the `CityJSON.add_to_j` method. See this in practice in the
[download](download.ipynb) tutorial.

The functions for loading and exporting from the API are `cjio.cityjson.load` and
`cjio.cityjson.save`.

In [5]:
from cjio import cityjson

p = (Path("data") / "sample.json").resolve()
cm = cityjson.load(path=p)

for co_id, co in cm.cityobjects.items():
    print(f"Found CityObject {co_id} of type {co.type} and instance of {type(co)}")

cityjson.save(citymodel=cm, path="outfile.json")

Found CityObject NL.IMBAG.Pand.0518100000285075 of type Building and instance of <class 'cjio.models.CityObject'>
Found CityObject NL.IMBAG.Pand.0518100000285075-0 of type BuildingPart and instance of <class 'cjio.models.CityObject'>
Found CityObject NL.IMBAG.Pand.0518100000346770 of type Building and instance of <class 'cjio.models.CityObject'>
Found CityObject NL.IMBAG.Pand.0518100000269458 of type Building and instance of <class 'cjio.models.CityObject'>
Found CityObject NL.IMBAG.Pand.0518100000346770-0 of type BuildingPart and instance of <class 'cjio.models.CityObject'>
Found CityObject NL.IMBAG.Pand.0518100000269458-0 of type BuildingPart and instance of <class 'cjio.models.CityObject'>


We can print the basic information about the city model.
Note that `print()` returns the same information as the `info` command in the CLI.

In [6]:
print(cm)

{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "bbox": [
    76538.049,
    451765.646,
    0.0,
    76614.34999999999,
    451874.674,
    11.987
  ],
  "transform/compressed": true,
  "cityobjects_total": 3,
  "cityobjects_present": [
    "Building",
    "BuildingPart"
  ],
  "materials": false,
  "textures": false
}


`print()` also works for CityObjects.

In [7]:
print(co)

{
  "id": "NL.IMBAG.Pand.0518100000269458-0",
  "type": "BuildingPart",
  "attributes": {},
  "children": [],
  "parents": [
    "NL.IMBAG.Pand.0518100000269458"
  ],
  "geometry_type": [
    "Solid"
  ],
  "geometry_lod": [
    1.2,
    2.2,
    1.3
  ],
  "semantic_surfaces": [
    "RoofSurface",
    "GroundSurface",
    "WallSurface"
  ]
}


Using the CLI methods, like `validate`. At the moment, the best if you browse the
source code of the [`cjio.cityjson.CityJSON` class](https://github.com/cityjson/cjio/blob/master/cjio/cityjson.py) to see how these methods work.

In [8]:
is_valid, wo_warnings, error_strings, warning_strings =  cm.validate()
print(f"\nThe city model is valid: {is_valid}")

-- Validating the syntax of the file
	(using the schemas 1.0.1)
-- Validating the internal consistency of the file (see docs for list)
	--Vertex indices coherent
	--Specific for CityGroups
	--Semantic arrays coherent with geometry
	--Root properties
	--Empty geometries
	--Duplicate vertices
	--Orphan vertices
	--CityGML attributes

The city model is valid: True


### Export to pandas DataFrame

It is possible to export the city model into a pandas DataFrame. Note that only the CityObject attributes are exported into the dataframe, with CityObject IDs as the index of the dataframe. Thus if you want to export the attributes of SemanticSurfaces for example, then you need to add them as CityObject attributes.

In [9]:
cm_df = cm.to_dataframe()
cm_df.head()

,dak_type,data_area,data_coverage,documentnummer,geconstateerd,gid,h_dak_50p,h_dak_70p,h_dak_max,h_dak_min,...,rmse_lod12,rmse_lod13,rmse_lod22,rn,status,t_run,val3dity_codes_lod12,val3dity_codes_lod13,val3dity_codes_lod22,voorkomenidentificatie
NL.IMBAG.Pand.0518100000285075,slanted,139.142502,0.938708,FV20101220-04,False,17625148.0,10.750,11.216,12.053,3.152,...,1.386077,1.092674,0.123877,1.0,Pand in gebruik,316.475006,[],[],[],1.0
NL.IMBAG.Pand.0518100000285075-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NL.IMBAG.Pand.0518100000346770,slanted,42.092503,0.872392,FV20101220-04,False,9075761.0,7.771,8.355,9.144,0.725,...,1.492180,1.376112,0.585438,1.0,Pand in gebruik,109.615997,[],[],[],1.0
NL.IMBAG.Pand.0518100000269458,horizontal,1.997500,0.248667,FP20101220-03,False,613043.0,2.882,2.896,2.951,2.816,...,0.035785,0.032785,0.030866,1.0,Pand in gebruik,36.872002,[],[],[],1.0
NL.IMBAG.Pand.0518100000346770-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
